# Wikipics async demo

## Referências sobre `asyncio` no Jupyter Notebook

* [IPython 7.0, Async REPL (from 2018)](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7)
* [Supporting asyncio.get_event_loop().run_until_complete() in repls](https://discuss.python.org/t/supporting-asyncio-get-event-loop-run-until-complete-in-repls/5573/9)
* [Running Asynchronous Code in Jupyter Notebooks: Managing Event Loops](https://medium.com/p/b9696a596ce4) (paywall, não vale o custo)

In [1]:
from pathlib import Path
from typing import NamedTuple

from IPython.display import display
from ipywidgets.widgets import Image, Layout
import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

Função que usamos na demonstração com threads e processos, denominada `fetch` em 
[wikipics-demo](wikipics-demo.ipynb).
Reproduzi aqui para comparar com a versão assíncrona de `fetch`, logo abaixo.

```python
def fetch_blocking(url) -> ImageRecord:
    resp = httpx.get(url)  # thread blocked here
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))
```

In [2]:
from httpx import AsyncClient

async def fetch(client:AsyncClient, url:str) -> ImageRecord:
    resp = await client.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

Outro trecho de código de [wikipics-demo](wikipics-demo.ipynb),
para comparação com o código usando `await` em seguida.

```python
url = get_sample_url(1_000_000)
img_rec = fetch_blocking(url)  # main thread blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)
```

In [3]:
url = get_sample_url(1_000_000)
async with AsyncClient() as client:
    img_rec = await fetch(client, url)  # console blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

   1_025_262 bytes | Aral_Sea_1989-2008.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\…

Mais um trecho de [wikipics-demo](wikipics-demo.ipynb), para comparação:

```python
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

for url in urls:
    img_rec = fetch(url)
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')
```


In [4]:
import asyncio

img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

async def download_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        results = await asyncio.gather(*tasks)
    return results

for img_rec in await download_pics(urls):
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')    

Image(value=b'\xff\xd8\xff\xe1$\tExif\x00\x00II*\x00\x08\x00\x00\x00\x0c\x00\x0e\x01\x02\x00\xbb\x00\x00\x00\x…

   1_972_894 bytes | Barack_Obama_family_portrait_2011.jpg


Image(value=b'\xff\xd8\xff\xe1%\xfeExif\x00\x00II*\x00\x08\x00\x00\x00\t\x00\x0f\x01\x02\x00\x06\x00\x00\x00z\…

   1_975_417 bytes | Dolceacqua43_-_Artista_locale_mentre_dipinge_un_acquarello.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00H\x00H\x00\x00\xff\xe1#\xb9Exif\x00\x00II*\x00\x…

...

   2_005_363 bytes | Marmot-edit1.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\x96\x00\x96\x00\x00\xff\xe1DvExif\x00\x00MM\x00…

   1_976_293 bytes | United_States_Declaration_of_Independence.jpg


Image(value=b'\xff\xd8\xff\xe1\x18hExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0c\x01\x00\x00\x03\x00\x00\x00\x01…

   1_969_876 bytes | Yogapith%2C_Mayapur.jpg
TOTAL BYTES: 19_973_936


In [5]:
# %%time
# Não funciona o %%time:
#   File <timed exec>:18
# SyntaxError: 'await' outside function

from gallery import Gallery
from httpx import HTTPStatusError

num_images = 10

urls = get_sample_urls(15_000_000, num_images)
gallery = Gallery(num_images)
gallery.display()

async def display_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        for i, coroutine in enumerate(asyncio.as_completed(tasks)):
            try:
                img_rec = await coroutine
            except HTTPStatusError:
                pass
            else:
                gallery.update(i, img_rec.pixels, img_rec.name)

await display_pics(urls)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)  14_541_999 bytes | Pahit-Pahit_Manis_pamphlet_%28obverse%29.jpg
( 2)  15_440_443 bytes | Vincent_van_Gogh_-_Sunflowers_-_VGM_F458.jpg
( 3)  14_608_619 bytes | St_James%27s_Church_Interior_2%2C_Spanish_Place%2C_London%2C_UK_-_Diliff.jpg
( 4)  15_217_910 bytes | USA_declaration_independence.jpg
( 5)  14_617_266 bytes | US-Colonial_%28MA-87.15%29-Massachusetts-1_May_1741_OBV.jpg
( 6)  15_247_075 bytes | Exterior_of_Blenduk_Church%2C_Semarang%2C_2014-06-18.jpg
( 7)  14_724_731 bytes | Wang_Ximeng._A_Thousand_Li_of_Rivers_and_Mountains._%28Complete%2C_51%2C3x1191%2C5_cm%29._1113._Palace_museum%2C_Beijing.jpg
( 8)  15_407_615 bytes | St_Etheldreda%27s_Church_Interior%2C_London%2C_UK_-_Diliff.jpg
( 9)  14_632_560 bytes | Red_telephone_box%2C_St_Paul%27s_Cathedral%2C_London%2C_England%2C_GB%2C_IMG_5182_edit.jpg
(10)  14_944_305 bytes | NI-125c-Netherlands_Indies-Japanese_Occupation-10_Gulden_%281942%29.jpg
TOTAL BYTES: 149_382_523
